In [7]:
import numpy as np
import os.path
import sys
from os import listdir
from os.path import isfile, join

In [8]:
def overlap(start1, end1, start2, end2):
    """Does the range (start1, end1) overlap with (start2, end2)?"""
    return end1 >= start2 and end2 >= start1

In [9]:
def time_ranges(file_name):
    file_labeled = open(file_name,"r")
    time_ranges = []
    for curr_line in file_labeled:
        curr_line_split = curr_line.split(" ")
        if curr_line_split[2] == "crosswalk\n":
            curr_range = [int(curr_line_split[0]), int(curr_line_split[1])]
            time_ranges.append(curr_range)
    return time_ranges

In [10]:
def eval_ranges(hand,auto):
    true_positive = 0
    false_positive = 0
    true_negative = 0
    
    for hand_row in hand:
        had_overlap = False
        for auto_row in auto:
            if overlap(hand_row[0],hand_row[1],auto_row[0],auto_row[1]):
                had_overlap = True
                break
        
        if had_overlap:
            true_positive = true_positive + 1
        else:
            true_negative = true_negative + 1
    
    for auto_row_2 in auto:
        had_overlap = False
        for hand_row_2 in hand:
            if overlap(hand_row_2[0],hand_row_2[1],auto_row_2[0],auto_row_2[1]):
                had_overlap = True
                break
        
        if not had_overlap:
            false_positive = false_positive + 1
    
    return true_positive,false_positive,true_negative

In [11]:
new_path = "/home/jspall/Documents/GitHub/ECE4271_Proj4/output"
hand_path = new_path+"/hand/"
auto_path = new_path+"/auto/"
os.chdir(new_path)


onlyfiles = [f for f in listdir(hand_path) if isfile(join(hand_path, f))]

txt_files = 0
hand_file_lists = []

for file in onlyfiles:
    if file.endswith(".txt") or file.endswith(".TXT"):
        txt_files = txt_files + 1
        hand_file_lists.append(file)
        
    else:
        print(file + " not txt, skipped.")

print("Found " +  str(txt_files) + " hand labeled files.")
if len(hand_file_lists) > 0:
    total_true_positives = 0
    total_false_positive = 0
    total_true_negative = 0
    
    for hand_file_row in hand_file_lists:
        current_hand_file_path = hand_path+hand_file_row
        current_auto_file_path = auto_path+hand_file_row
        
        hand_ranges = time_ranges(current_hand_file_path)
        
        if(not os.path.isfile(current_auto_file_path)):
            all_true_negatives = len(hand_ranges)
            total_true_negative = total_true_negative + all_true_negatives
            print("Missing " + hand_file_row + " in auto for comparison, marking " + str(all_true_negatives) 
                  + " as true negatives.")
        else:
            auto_ranges = time_ranges(current_auto_file_path)
            curr_tp,curr_fp,curr_tn = eval_ranges(hand_ranges,auto_ranges)
            total_true_positives = total_true_positives + curr_tp
            total_false_positive = total_false_positive + curr_fp
            total_true_negative = total_true_negative + curr_tn
            print(hand_file_row + " results: " 
                  + str(curr_tp) + " true positives, " 
                  + str(curr_fp) + " false positives, "
                  + str(curr_tn) + " true negatives")
            
    
        
    print("Total true positives: " + str(total_true_positives))
    print("Total false positives: " + str(total_false_positive))
    print("Total true negatives: " + str(total_true_negative))
    print("Total accuracy: " + 
          str(100*total_true_positives/(total_true_positives+total_false_positive+total_true_negative)) + "%")
else:
    print("Missing hand labeled files!")
print("Finished crosswalk evaluation")

Found 5 hand labeled files.
5E64065F.txt results: 9 true positives, 4 false positives, 0 true negatives
5E63EB7D.txt results: 6 true positives, 7 false positives, 16 true negatives
5E6409C3.txt results: 11 true positives, 4 false positives, 0 true negatives
5E63FC33.txt results: 10 true positives, 0 false positives, 4 true negatives
5E640A7D.txt results: 8 true positives, 17 false positives, 2 true negatives
Total true positives: 44
Total false positives: 32
Total true negatives: 22
Total accuracy: 44.89795918367347%
Finished crosswalk evaluation
